# Random Shaking Volume Denoising (RSVD) using Farneback3D (small)

In [1]:
use_opticalflow3D = False

In [2]:
vol_name = "small_vol"

In [3]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    print("Running in Colab")
    !pip install cupy-cuda12x
    !pip install opticalflow3D
    !apt install libcudart11.0
    !apt install libcublas11
    !apt install libcufft10
    !apt install libcusparse11
    !apt install libnvrtc11.2
    from google.colab import drive
    drive.mount('/content/drive')
    !cp drive/Shareddrives/TomogramDenoising/tomograms/{vol_name}.tif .
else:
    print("Running in locahost")
    !cp ~/Downloads/{vol_name}.tif .

Running in locahost
cp: cannot stat '/home/vruiz/Downloads/small_vol.tif': No such file or directory


In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [6]:
import motion_estimation # pip install "motion_estimation @ git+https://github.com/vicente-gonzalez-ruiz/motion_estimation"

In [7]:
import information_theory  # pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"

In [8]:
if use_opticalflow3D:
    import opticalflow3D
    import warnings
    from numba.core.errors import NumbaPerformanceWarning
    import numpy as np
    
    warnings.filterwarnings("ignore", category=NumbaPerformanceWarning)
else:
    import motion_estimation._3D.farneback as farneback3D

In [9]:
import mrcfile
from collections import namedtuple

In [10]:
Args = namedtuple("args", "input")
args = Args("small_vol.mrc")

In [11]:
%%bash -s "$args.input"
set -x
OUTPUT_FILENAME=$1
#rm -f $OUTPUT_FILENAME
if test ! -f $OUTPUT_FILENAME ; then
    FILEID="1YDyHWyB1d6yq2LYif2zQcQgr7vhlspYo"
    wget --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O $OUTPUT_FILENAME 2> /dev/null
fi
set +x

+ OUTPUT_FILENAME=small_vol.mrc
+ test '!' -f small_vol.mrc
+ set +x


In [12]:
stack_MRC = mrcfile.open(args.input)
noisy = stack_MRC.data[0:32,...]

In [14]:
import volume.RSM

ModuleNotFoundError: No module named 'opticalflow3D'

In [ ]:
noisy = (255*(noisy - np.min(noisy))/(np.max(noisy) - np.min(noisy))).astype(np.uint8)

In [ ]:
if use_opticalflow3D:
    block_size = (noisy.shape[0], noisy.shape[1], noisy.shape[2])

In [ ]:
if use_opticalflow3D:
    #farneback = opticalflow3D.Farneback3D(iters=5, num_levels=3, scale=0.5, spatial_size=33, presmoothing=None, filter_type="gaussian", filter_size=3)
    #farneback = opticalflow3D.Farneback3D(iters=5, num_levels=3, scale=0.5, spatial_size=13, presmoothing=None, filter_type="gaussian", filter_size=7)
    #farneback = opticalflow3D.Farneback3D(iters=5, num_levels=3, scale=0.5, spatial_size=5, presmoothing=3, filter_type="gaussian", filter_size=5)
    #farneback = opticalflow3D.Farneback3D(iters=5, num_levels=3, scale=0.5, spatial_size=3, presmoothing=5, filter_type="box", filter_size=3)
    #farneback = opticalflow3D.Farneback3D(iters=5, num_levels=5, scale=0.5, spatial_size=3, presmoothing=5, filter_type="box", filter_size=3)
    #farneback = opticalflow3D.Farneback3D(iters=5, num_levels=5, scale=0.5, spatial_size=3, presmoothing=5, filter_type="gaussian", filter_size=11)
    #farneback = opticalflow3D.Farneback3D(iters=5, num_levels=1, scale=0.5, spatial_size=5, presmoothing=4, filter_type="box", filter_size=5)
    estimator = opticalflow3D.Farneback3D(iters=5, num_levels=5, scale=0.5, spatial_size=9, sigma_k=0.15, filter_type="gaussian" ,filter_size=7, presmoothing=None, device_id=0)
else:
    estimator = farneback3D(

In [ ]:
RS_sigma = 0.75
if use_opticalflow3D:
    denoised = RSIVD.filter(farneback, block_size, noisy, RS_sigma=RS_sigma, N_iters=20)
else:
    

In [ ]:
np.min(denoised)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 32))
axs[0].imshow(noisy[16], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised[16], cmap="gray")
axs[1].set_title(f"Denoised (DQI={information_theory.information.compute_quality_index(noisy[16], denoised[16])})")
fig.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 32))
axs[0].imshow(noisy[:, 100], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised[:, 100], cmap="gray")
axs[1].set_title(f"Denoised (DQI={information_theory.information.compute_quality_index(noisy[:, 100], denoised[:, 100])})")
fig.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 32))
axs[0].imshow(noisy[:, :, 100], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised[:, :, 100], cmap="gray")
axs[1].set_title(f"Denoised (DQI={information_theory.information.compute_quality_index(noisy[:, :, 100], denoised[:, :, 100])})")
fig.tight_layout()
plt.show()

In [ ]:
figure(figsize=(32, 32))
plt.subplot(1, 3, 1)
plt.title("original")
imgplot = plt.imshow(noisy_vol[15][::-1, :], cmap="gray")
plt.subplot(1, 3, 2)
plt.title("$\sigma_\mathrm{RS}=$"+f"{RS_sigma}")
plt.imshow(denoised_vol[15][::-1, :], cmap="gray")
plt.subplot(1, 3, 3)
plt.title("difference")
plt.imshow(noisy_vol[15][::-1, :] - denoised_vol[15][::-1, :], cmap="gray")

In [ ]:
figure(figsize=(32, 32))
plt.subplot(1, 3, 1)
plt.title("original")
imgplot = plt.imshow(noisy_vol[15][::-1, :], cmap="gray")
plt.subplot(1, 3, 2)
plt.title("$\sigma_\mathrm{RS}=$"+f"{RS_sigma}")
plt.imshow(denoised_vol[15][::-1, :], cmap="gray")
plt.subplot(1, 3, 3)
plt.title("difference")
plt.imshow(noisy_vol[15][::-1, :] - denoised_vol[15][::-1, :], cmap="gray")

In [ ]:
skimage.io.imsave(f"{vol_name}_denoised_{RS_sigma}.tif", denoised_vol, imagej=True)

In [ ]:
img = skimage.io.imread("http://www.hpca.ual.es/~vruiz/images/barb.png")#[:256, :256]

In [ ]:
noisy_vol = np.stack([img]*32)

In [ ]:
noisy_vol.shape

In [ ]:
mean = 0
var = 1000
sigma = var**0.5
for i in range(noisy_vol.shape[0]):
    noise = np.random.normal(mean,sigma,img.shape).reshape(img.shape)
    noisy_vol[i] = np.clip(a=img.astype(np.float32) + noise, a_min=0, a_max=255).astype(np.uint8)

In [ ]:
farneback = opticalflow3D.Farneback3D(iters=5,
                                      num_levels=3,
                                      scale=0.5,
                                      spatial_size=5,
                                      presmoothing=2,
                                      filter_type="box",
                                      filter_size=5,
                                     )

In [ ]:
RS_sigma = 1.0
denoised_vol = RSIVD.filter(farneback, block_size, noisy_vol, RS_sigma=RS_sigma, N_iters=30)

In [ ]:
figure(figsize=(32, 32))
plt.subplot(1, 3, 1)
plt.title("original")
imgplot = plt.imshow(noisy_vol[15], cmap="gray")
plt.subplot(1, 3, 2)
plt.title("$\sigma_\mathrm{RS}=$"+f"{RS_sigma}")
plt.imshow(denoised_vol[15], cmap="gray")
plt.subplot(1, 3, 3)
plt.title("difference")
plt.imshow(noisy_vol[15] - denoised_vol[15], cmap="gray")

In [ ]:
RS_sigma = 1.0
denoised_vol = RSIVD.filter(farneback, block_size, noisy_vol, RS_sigma=RS_sigma, N_iters=100)

In [ ]:
figure(figsize=(32, 32))
plt.subplot(1, 3, 1)
plt.title("original")
imgplot = plt.imshow(noisy_vol[15], cmap="gray")
plt.subplot(1, 3, 2)
plt.title("$\sigma_\mathrm{RS}=$"+f"{RS_sigma}")
plt.imshow(denoised_vol[15], cmap="gray")
plt.subplot(1, 3, 3)
plt.title("difference")
plt.imshow(noisy_vol[15] - denoised_vol[15], cmap="gray")

In [ ]:
RS_sigma = 2.0
denoised_vol = RSIVD.filter(farneback, block_size, noisy_vol, RS_sigma=RS_sigma, N_iters=30)

In [ ]:
figure(figsize=(32, 32))
plt.subplot(1, 3, 1)
plt.title("original")
imgplot = plt.imshow(noisy_vol[15], cmap="gray")
plt.subplot(1, 3, 2)
plt.title("$\sigma_\mathrm{RS}=$"+f"{RS_sigma}")
plt.imshow(denoised_vol[15], cmap="gray")
plt.subplot(1, 3, 3)
plt.title("difference")
plt.imshow(noisy_vol[15] - denoised_vol[15], cmap="gray")

In [ ]:
RS_sigma = 3.0
denoised_vol = RSIVD.filter(farneback, block_size, noisy_vol, RS_sigma=RS_sigma, N_iters=30)

In [ ]:
figure(figsize=(32, 32))
plt.subplot(1, 3, 1)
plt.title("original")
imgplot = plt.imshow(noisy_vol[15], cmap="gray")
plt.subplot(1, 3, 2)
plt.title("$\sigma_\mathrm{RS}=$"+f"{RS_sigma}")
plt.imshow(denoised_vol[15], cmap="gray")
plt.subplot(1, 3, 3)
plt.title("difference")
plt.imshow(noisy_vol[15] - denoised_vol[15], cmap="gray")

In [ ]:
#skimage.io.imsave(f"{vol_name}_denoised_{RS_sigma}.tif", denoised_vol, imagej=True)